In [ ]:


import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
import numpy as np
import pickle as pkl
import pickle
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from YahooDataReader import YahooDataReader
import math
import statistics
import itertools
from random import seed, shuffle
import os,sys
import urllib.request  as urllib2
from collections import defaultdict
from sklearn import preprocessing
from IPython.display import display
from collections import Counter

In [ ]:
def KL_Divergence(p1,p2,q1,q2): #can use jenson divergance that is bounded between 0 and 1#
    if p1!=0 and p2!=0:
        J=(    (p1*np.log((2*p1)/(p1+q1)))  +  (p2*np.log((2*p2)/(p2+q2)))    +  (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))    ) / 2
    elif p1==0:
        J=(    (p2*np.log((2*p2)/(p2+q2)))    +  (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))    ) / 2
    elif p2==0:
        J=(   (p1*np.log((2*p1)/(p1+q1)))  +  (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))    ) / 2
    else:
        J=(   (q1*np.log((2*q1)/(p1+q1)))  +  (q2*np.log((2*q2)/(p2+q2)))   )/ 2 
    return J

In [ ]:
def fairness(size_docID, 
             size_female, 
             size_male, 
             current_male_CTR, 
             sofar_male_CTR, 
             current_female_CTR, 
             sofar_female_CTR,
             ):


    CTR_male = sofar_male_CTR + current_male_CTR
    CTR_female =  sofar_female_CTR + current_female_CTR
    CTR_total = CTR_male + CTR_female
    current_fairness= 1- KL_Divergence(CTR_male / CTR_total,
                                       CTR_female / CTR_total,
                                       size_male/size_docID, 
                                       size_female/size_docID
                                       )
    return current_fairness

In [ ]:
def separate_and_sort(
    docID,
    rel_scores
    ):
    """
    Arg list:
    movieID: list of movie IDs. Here, list of all docID for a certain qid
    rel_scores: dict() that maps movie_ID -> rel_score
    """
    female_sorted_by_rel = []
    male_sorted_by_rel = []
    
    for i in docID:    
        if i in FemaleID_str:
            female_sorted_by_rel.append(i)
        else:
            male_sorted_by_rel.append(i)

    female_sorted_by_rel.sort(key = lambda x : rel_scores[x], reverse=True) 
    male_sorted_by_rel.sort(key = lambda x : rel_scores[x], reverse=True)

    return male_sorted_by_rel, female_sorted_by_rel

In [ ]:

def interpolation_optimized(
    qid,
    docID,
    rel_scores,
    Z,
):
    """
    Arg list:
    docID: list of docIDs for each qid in test data
    rel_scores: dict() that maps doc_ID -> rel_score
    Z: Z value for interpolation
    """
    all_sorted_by_rel= sorted(docID, key = lambda x : rel_scores[x], reverse=True)
    S=[]
    sofar_female_CTR=0
    sofar_male_CTR=0
    sofar_DCG=0
    IDCG=0
    
    availablity = all_sorted_by_rel[:] #make a copy of all_sorted to avoid del make problem for IDCG
    while len (S)< 30:
        IDCG= IDCG+ (float(2**float(rel_scores[all_sorted_by_rel[len(S)]])-1) /  math.log2(1+len(S)+1))
        epsilon_1= 0.65
        epsilon_plus= 1- ((len(S)+2)/100)
        epsilon_minus= epsilon_1* (1/min( len(S)+1, 10) )
        max_intpol_score = 0
        max_item_data = None
        for item in availablity:
            DCG = sofar_DCG+ (float(2**float(rel_scores[item])-1) /  math.log2(1+len(S)+1))
            nDCG =  float (DCG)/ IDCG
            if item in FemaleID_str:
                current_male_CTR = 0
                current_female_CTR = float(  (rel_scores[item]* epsilon_plus) + ( (1-rel_scores[item]) * epsilon_minus)  ) * 1/math.log2(1+len(S)+1)
            else:
                current_male_CTR =   float(  (rel_scores[item]* epsilon_plus) + ( (1-rel_scores[item]) * epsilon_minus)  ) * 1/math.log2(1+len(S)+1)
                current_female_CTR = 0
            fair_metric = fairness(sum_rel_totall_predicted[qid], sum_rel_female_predicted[qid], sum_rel_male_predicted[qid], current_male_CTR, sofar_male_CTR, current_female_CTR, sofar_female_CTR)
            intpol_score = (1-Z) * nDCG + Z * fair_metric
            if intpol_score > max_intpol_score:
                max_intpol_score = intpol_score
                max_item_data = (item, current_female_CTR, current_male_CTR, DCG)



        S.append(max_item_data[0])
        availablity.remove(max_item_data[0])
        sofar_female_CTR += max_item_data[1]
        sofar_male_CTR += max_item_data[2]
        sofar_DCG = max_item_data[3]
        
    return S

In [ ]:

def get_nDCG(docID, rel_scores, item_list):
    """
    gives the nDCG of ranking
    input:
    docID: id of documents
    rel_scores: relevant scores assuming predict = ground truth or not
    item_list: dictionary key= docID, value= relscores
    """
    sorted_docID= sorted(docID, key=lambda x:rel_scores [int(x)] , reverse=True) #for ML1M
    #print ("sorted_docID", sorted_docID)
    Denom= float(0)
    Nom= float(0)
    for i in range (len(item_list)):
        temp1= D_real[str(sorted_docID[i])] #D1 if want to assume predicted rel is ground truth, D_real otherwise#
        temp2= 2**(float (temp1))
        Denom= Denom+(   (temp2-1)  / (math.log2(i+2))    )
        temp3= D_real[str(item_list[i])]    #D1 if want to assume predicted rel is ground truth, D_real otherwise#
        temp4= 2**(float (temp3))
        Nom=Nom + (  (temp4-1)  / (math.log2(i+2))    )
    nDCG= (float(Nom)/float(Denom))
    return (nDCG, sorted_docID  )

In [ ]:
def get_divergence (qid, docID, rel_scores):
    sofar_female_CTR=0
    sofar_male_CTR=0
    D_movie_list=[]
    D_male_list=[]
    D_female_list=[]
    for i in range(len(docID)):
        epsilon_1= 0.65
        epsilon_plus= 1- ((i+2)/100)
        epsilon_minus= epsilon_1* ( 1/min (i+1, 10) )
        if docID[i] in FemaleID_str:
            sofar_female_CTR += float(  (rel_scores[docID[i]]* epsilon_plus) + ( (1-rel_scores[docID[i]]) * epsilon_minus)  ) * 1/math.log2(i+2)
        else:
            sofar_male_CTR +=   float(  (rel_scores[docID[i]]* epsilon_plus) + ( (1-rel_scores[docID[i]]) * epsilon_minus)  ) * 1/math.log2(i+2)
        
        CTR_total= sofar_female_CTR+ sofar_male_CTR  
        D_movie_list.append(KL_Divergence(sofar_male_CTR/ CTR_total, sofar_female_CTR/ CTR_total , sum_rel_male[qid]/sum_rel_totall[qid], sum_rel_female[qid]/sum_rel_totall[qid]))
        D_male_list.append(float((sofar_male_CTR/ CTR_total)/(sum_rel_male[qid]/sum_rel_totall[qid])))
        D_female_list.append(float((sofar_female_CTR/ CTR_total)/(sum_rel_female[qid]/sum_rel_totall[qid])))
    return (D_movie_list, D_male_list, D_female_list)

In [ ]:
def get_divergence_trust_DP (qid, docID, rel_scores):
    sofar_female_CTR=0
    sofar_male_CTR=0
    D_movie_list=[]
    for i in range(len(docID)):
        epsilon_1= 0.65
        epsilon_plus= 1- ((i+2)/100)
        epsilon_minus= epsilon_1* ( 1/min (i+1, 10) )
        if docID[i] in FemaleID_str:
            sofar_female_CTR += float(  (rel_scores[docID[i]]* epsilon_plus) + ( (1-rel_scores[docID[i]]) * epsilon_minus)  ) * 1/math.log2(i+2)
        else:
            sofar_male_CTR +=   float(  (rel_scores[docID[i]]* epsilon_plus) + ( (1-rel_scores[docID[i]]) * epsilon_minus)  ) * 1/math.log2(i+2)
        
        CTR_total= sofar_female_CTR+ sofar_male_CTR  
        D_movie_list.append(KL_Divergence(sofar_male_CTR/ CTR_total, sofar_female_CTR/ CTR_total , size_male_test[qid]/totall_size, size_female_test[qid]/totall_size))
    return (D_movie_list)

In [ ]:
def get_divergence_pos_DT (qid, docID, rel_scores):
    sofar_female_exp=0
    sofar_male_exp=0
    D_movie_list=[]
    for i in range(len(docID)):
        if docID[i] in FemaleID_str:
            sofar_female_exp +=  1/math.log2(i+2)
        else:
            sofar_male_exp +=  1/math.log2(i+2)
        
        exp_total= sofar_female_exp+ sofar_male_exp  
        D_movie_list.append(KL_Divergence(sofar_male_exp/ exp_total, sofar_female_exp/ exp_total , sum_rel_male[qid]/sum_rel_totall[qid], sum_rel_female[qid]/sum_rel_totall[qid]))
    return (D_movie_list)

In [ ]:
def get_divergence_pos_DP (qid, docID, rel_scores):
    sofar_female_exp=0
    sofar_male_exp=0
    D_movie_list=[]
    for i in range(len(docID)):
        if docID[i] in FemaleID_str:
            sofar_female_exp +=  1/math.log2(i+2)
        else:
            sofar_male_exp +=  1/math.log2(i+2)
        
        exp_total= sofar_female_exp+ sofar_male_exp  
        D_movie_list.append(KL_Divergence(sofar_male_exp/ exp_total, sofar_female_exp/ exp_total , size_male_test[qid]/totall_size, size_female_test[qid]/totall_size))
    return (D_movie_list)

In [ ]:
#This function is fairness definition of joachims (difference) based on disparate treatment

def get_joachims_diff (qid, docID):  #gives a vector that has conv at each rank indx. can feed both item_list and sorted_item_list#
    
        
    sofar_female_expo=0
    sofar_male_expo=0
    sofar_indx_expo=0
    Diff_movie_list=[]
    for i in range(len(docID)):
        if docID[i] in FemaleID_str:
            sofar_female_expo= sofar_female_expo+(1/math.log2(i+2)) 
        else:
            sofar_male_expo= sofar_male_expo + (1/math.log2(i+2))   
        sofar_indx_expo= sofar_indx_expo+ (1/math.log2(i+2))
        if sum_rel_male[qid]/size_male_test[qid] > sum_rel_female[qid]/size_female_test[qid]:
            sign= 1
        else:
            sign=-1
        Diff_movie_list.append(max(0, sign*diff(sofar_male_expo/ size_male_test[qid], sofar_female_expo/ size_female_test[qid] , sum_rel_male[qid]/size_male_test[qid], sum_rel_female[qid]/size_female_test[qid])))

    return (Diff_movie_list)

In [ ]:
def get_CPFair_diff (qid, docID):  #gives a vector that has conv at each rank indx. can feed both item_list and sorted_item_list#
    
        
    sofar_female_count=0
    sofar_male_count=0
    Diff_movie_list=[]
    for i in range(len(docID)):
        if docID[i] in FemaleID_str:
            sofar_female_count= sofar_female_count+1 
        else:
            sofar_male_count= sofar_male_count + 1   
        Diff_movie_list.append(abs(sofar_male_count- sofar_female_count))

    return (Diff_movie_list)

In [ ]:
def get_FairMetric(DocID, DivOrDiff_item_list):    #do not need it if we have new trust and CTR
    Denom=float(0)
    Nom=float(0)
    for i in range (len(DocID)):
        Denom= Denom+ (1/ (math.log2(i+2)))
        Nom= Nom+ (1/ (math.log2(i+2)))*DivOrDiff_item_list[i]   
    n_item_list= float(Nom)/ float(Denom)
    return (n_item_list)

In [ ]:

import math

interplotion_value=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.91, 0.92, 0.94, 0.96, 0.98, 0.99, 1]
for z in interplotion_value:
    D2={} # a dictionary: key= user id, value= S/item_list

    nDCG_all_q=[]
    D_item_list_all_q=[]
    male_Div_all_q=[]
    female_Div_all_q=[]
    
    D_item_list_trust_DP_all_q=[]
    D_item_list_pos_DT_all_q=[]
    D_item_list_pos_DP_all_q=[]
    joachims_diff_item_list_all_q=[]
    CPFair_diff_item_list_all_q=[]
    

    



    
    
    for qid in range(len(docID_test)):
        nDCG=[]
        nDiv=[]
        D1={} # a dictionary: key= movieID, value= predicted rel_scores
        D_real= {} # a dictionary: key= movieID, value= true rel_scores
        for i in range (len(docID_test[0])):
            #D1[docID_test_str[qid][i]]= predicted_rels[qid][i] # for German Compas Comcast and ML100K
            #D_real[docID_test_str[qid][i]]= data_test[1][qid][i] # for German Compas Comcast and ML100K
            
            D1[docID_test_str[qid][i]]= predicted_rels[qid][int (float(docID_test_str[qid][i]))] # for ML1M
            D_real[docID_test_str[qid][i]]= data_test[1][qid][int (float(docID_test_str[qid][i]))] # for ML1M

        item_list  = interpolation_optimized (qid,docID_test_str[qid], D1, z)
        for i in range(len(item_list)):
            nDCG.append(get_nDCG(docID_test[qid], data_test[1][qid], item_list[0:i+1])[0]) #predicted_rels[qid] if assume precietd rel is ground truth, data_test[1][qid] Otherwise
            D_item_list= get_divergence(qid, item_list[0:i+1], D_real)[0]  #D1 if we assume predicted=true rel, and D_real o.w
            male_Div= get_divergence(qid, item_list[0:i+1], D_real)[1]
            female_Div= get_divergence(qid, item_list[0:i+1], D_real)[2]
            
            D_item_list_trust_DP= get_divergence_trust_DP (qid, item_list[0:i+1], D_real)
            D_item_list_pos_DT= get_divergence_pos_DT (qid, item_list[0:i+1], D_real)
            D_item_list_pos_DP= get_divergence_pos_DP (qid, item_list[0:i+1], D_real)
            joachims_diff_item_list= get_joachims_diff (qid, item_list[0:i+1])
            CPFair_diff_item_list= get_CPFair_diff (qid, item_list[0:i+1])
            
            


        nDCG_all_q.append (nDCG)
        D_item_list_all_q.append(D_item_list)
        male_Div_all_q.append(male_Div)
        female_Div_all_q.append(female_Div)
        
        D_item_list_trust_DP_all_q.append(D_item_list_trust_DP)
        D_item_list_pos_DT_all_q.append(D_item_list_pos_DT)
        D_item_list_pos_DP_all_q.append(D_item_list_pos_DP)
        joachims_diff_item_list_all_q.append(joachims_diff_item_list)
        CPFair_diff_item_list_all_q.append(CPFair_diff_item_list)

        


    avg_nDCG_at_indx_all_q= np.mean(nDCG_all_q, axis=0)
    avg_D_item_list_at_indx_all_q= np.mean(D_item_list_all_q, axis=0)
    avg_male_Div_at_indx_all_q= np.mean(male_Div_all_q, axis=0)
    avg_female_Div_at_indx_all_q= np.mean(female_Div_all_q, axis=0)
    
    avg_D_item_list_trust_DP_at_indx_all_q= np.mean(D_item_list_trust_DP_all_q, axis=0)
    avg_D_item_list_pos_DT_at_indx_all_q= np.mean(D_item_list_pos_DT_all_q, axis=0)
    avg_D_item_list_pos_DP_at_indx_all_q= np.mean(D_item_list_pos_DP_all_q, axis=0)
    avg_joachims_diff_item_list_at_indx_all_q= np.mean(joachims_diff_item_list_all_q, axis=0)
    avg_CPFair_diff_item_list_at_indx_all_q= np.mean(CPFair_diff_item_list_all_q, axis=0)
    

    print ("K=5")
    print (avg_nDCG_at_indx_all_q[4], end='\t')
    print (avg_D_item_list_at_indx_all_q[4], end='\t')
    print (avg_male_Div_at_indx_all_q[4], end='\t')
    print (avg_female_Div_at_indx_all_q[4], end='\t')
    print (avg_D_item_list_trust_DP_at_indx_all_q[4], end='\t')
    print (avg_D_item_list_pos_DT_at_indx_all_q[4], end='\t')
    print (avg_D_item_list_pos_DP_at_indx_all_q[4], end='\t')
    print (avg_joachims_diff_item_list_at_indx_all_q[4], end='\t')
    print (avg_CPFair_diff_item_list_at_indx_all_q[4], end='\t')
    print ("********************")
    print ("********************")
    print ("K=10")
    print ( avg_nDCG_at_indx_all_q[9], end='\t')
    print ( avg_D_item_list_at_indx_all_q[9], end='\t')
    print ( avg_male_Div_at_indx_all_q[9], end='\t')
    print ( avg_female_Div_at_indx_all_q[9], end='\t')
    print (avg_D_item_list_trust_DP_at_indx_all_q[9], end='\t')
    print (avg_D_item_list_pos_DT_at_indx_all_q[9], end='\t')
    print (avg_D_item_list_pos_DP_at_indx_all_q[9], end='\t')
    print ( avg_joachims_diff_item_list_at_indx_all_q[9], end='\t')
    print ( avg_CPFair_diff_item_list_at_indx_all_q[9], end='\t')
    print ("********************")
    print ("********************")
    print ("K=30")
    print ( avg_nDCG_at_indx_all_q[29], end='\t')
    print ( avg_D_item_list_at_indx_all_q[29], end='\t')
    print ( avg_male_Div_at_indx_all_q[29], end='\t')
    print ( avg_female_Div_at_indx_all_q[29], end='\t')
    print ( avg_D_item_list_trust_DP_at_indx_all_q[29], end='\t')
    print ( avg_D_item_list_pos_DT_at_indx_all_q[29], end='\t')
    print ( avg_D_item_list_pos_DP_at_indx_all_q[29], end='\t')
    print ( avg_joachims_diff_item_list_at_indx_all_q[29], end='\t')
    print ( avg_CPFair_diff_item_list_at_indx_all_q[29], end='\t')
    print ("********************")
    print ("********************")